In [1]:
import numpy as np
import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import datasets, models, transforms
import sys
from tqdm import tqdm
from wiptrashymodules.dummydataloader import load_data, split_train_val_test
from wiptrashymodules.dummyviz import show_confusion_mat, imshow, create_grid_for_mb
import matplotlib.pyplot as plt

In [2]:
class VGG(object):

    def __init__(self, pretrained_model, device, num_classes=25, lr=0.0001, reg=0.0, dtype=np.float32, mode="ft_extract"):
        self.params = {}
        self.reg = reg
        self.dtype = dtype 
        self.model = pretrained_model
        self.num_classes = num_classes
        self.lr = lr
        self.loss_fn = nn.CrossEntropyLoss()
        self.device = device
        self.save_model_path = '../wipTrashy/trainedModel2_VGG19.pt'

        self.set_parameter_requires_grad(mode)
        num_features = self.model.classifier[6].in_features
        features = list(self.model.classifier.children())[:-1]                  
        features.extend([nn.Linear(num_features, num_classes).to(self.device)]) 
        self.model.classifier = nn.Sequential(*features)            
                            
    def set_parameter_requires_grad(self, mode):
        if mode == "ft_extract":
            for param in self.model.features.parameters():
                param.requires_grad = False
        elif mode == "finetune_last":
            for param in self.model.features[:19].parameters():
                param.requires_grad = False
        
                
    def gather_optimizable_params(self):
        params_to_optimize = []
        for name, param in self.model.named_parameters():
            if param.requires_grad == True:
                params_to_optimize.append(param)

        return params_to_optimize

    
    def train(self, dataloaders, dataset_sizes, num_epochs):
        best_model_wts = copy.deepcopy(self.model.state_dict())
        best_acc = 0.0

        params_to_optimize = self.gather_optimizable_params()
        optimizer = optim.Adam(params_to_optimize, lr = self.lr)
        exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

        for epoch in tqdm(range(0, num_epochs)):
            print("Epoch {}/{}".format(epoch, num_epochs-1))
            print('-'*10)
            
            for mode in ['train', 'val']:
                if mode == "train":
                    exp_lr_scheduler.step()
                    self.model.train()
                else:
                    self.model.eval() 
                    
                total_loss = 0.0
                total_correct = 0 

                for inputs, labels in dataloaders[mode]:
                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)
                    
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(mode == 'train'):
                        outputs = self.model(inputs)
                        _, y_preds = torch.max(outputs, 1)

                        loss = self.loss_fn(outputs, labels)
                
                        if mode == "train":
                            loss.backward() 
                            optimizer.step()
                
                    total_loss += loss.item() * inputs.size(0)
                    total_correct += torch.sum(y_preds == labels.data)
                
                epoch_loss = total_loss / dataset_sizes[mode]
                epoch_acc = total_correct.double() / dataset_sizes[mode]

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(mode, epoch_loss, epoch_acc))
            
                if mode == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(self.model.state_dict())
                    torch.save(self.model.state_dict(), self.save_model_path)
                    
            print()

        print('Best val Acc: {:4f}'.format(best_acc))

        self.model.load_state_dict(best_model_wts)

        torch.save(self.model.state_dict(), self.save_model_path)
        
        return self.model



    def eval_model(self, dataloaders, mode = 'val'):
        since = time.time()
        avg_loss, avg_acc, total_loss, total_correct = 0,0,0,0
        num_batches = len(dataloaders[mode])
        mode_str = "Validation" if mode == 'val' else "Test"
        
        print("Evaluating model on {} set".format(mode_str))
        print('-' * 10)

        with torch.no_grad():
            for i, data in enumerate(dataloaders[mode]):
                if i % 100 == 0:
                    print("\r{} batch {}/{}".format(mode_str, i, num_batches), end='', flush=True)
                
                self.model.train(False)
                self.model.eval()

                inputs, labels = data
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                
                outputs = self.model(inputs)
                
                _, preds = torch.max(outputs.data, 1)
                loss = self.loss_fn(outputs, labels)
                
                total_loss += loss.item() * inputs.size(0)
                total_correct += torch.sum(preds == labels.data)
            
                del inputs, labels, outputs, preds
                torch.cuda.empty_cache()
            
        avg_loss = total_loss / dataset_sizes[mode]
        avg_acc = total_correct.double() / dataset_sizes[mode]
            
        elapsed_time = time.time() - since
        print()
        print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
        print("Average {} loss     : {:.4f}".format(mode_str, avg_loss))
        print("Average {} accuracy : {:.4f}".format(mode_str, avg_acc))
        print('-' * 10)

                
                
    def load_model(self, path, train_mode = False):
        self.model.load_state_dict(torch.load(path))
        self.model.to(self.device)

        if train_mode == False:
            self.model.eval()

        return self.model


    def visualize_model(self, dataloaders, num_images=16):
        self.model.train(False)
        self.model.eval()
        
        images_so_far = 0
        file_path_base = '../wipTrashy/'
        confusion_matrix = torch.zeros(self.num_classes, self.num_classes)
                                                   
        with torch.no_grad():
            for i, data in enumerate(dataloaders['test']):
                inputs, labels = data
                size = inputs.size()[0]
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = self.model(inputs)                
                _, preds = torch.max(outputs, 1)

                for t, p in zip(labels.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
                    
                create_grid_for_mb(i, inputs, num_images, class_names, preds, labels, file_path_base)
        show_confusion_mat(confusion_matrix, self.num_classes, class_names, outfile=file_path_base + "confusion_matrix_vgg19.jpg")

In [3]:
from wiptrashymodules.dummydataloader import load_data, split_train_val_test
split_train_val_test()

In [3]:
pathname = '../wipTrashy/trashyTrainTestVal1'
dataloaders, dataset_sizes, class_names = load_data(pathname)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg19 = models.vgg19(weights='VGG19_Weights.DEFAULT').to(device)

In [5]:
vgg_model = VGG(vgg19, device, num_classes=25, mode="finetune_all")

In [6]:
vgg_model.load_model('../wipTrashy/trainedModel_VGG19.pt', train_mode = False)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [7]:
vgg_model.train(dataloaders, dataset_sizes, num_epochs=150)

  0%|          | 0/150 [00:00<?, ?it/s]C:\Users\avani\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 0/149
----------
train Loss: 0.4615 Acc: 0.8672
val Loss: 0.3748 Acc: 0.8981


  1%|          | 1/150 [1:01:10<151:54:05, 3670.10s/it]


Epoch 1/149
----------
train Loss: 0.4351 Acc: 0.8750


  1%|▏         | 2/150 [2:02:33<151:12:55, 3678.22s/it]

...

 99%|█████████▉| 149/150 [155:04:54<1:07:00, 4020.34s/it]

val Loss: 0.2355 Acc: 0.9392

Epoch 149/149
----------
train Loss: 0.1615 Acc: 0.9538


100%|██████████| 150/150 [156:12:50<00:00, 3749.14s/it]  

val Loss: 0.2355 Acc: 0.9392

Best val Acc: 0.939797


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [12]:
vgg_model.load_model('../wipTrashy/trainedModel2_VGG19.pt', train_mode = False)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
vgg_model.visualize_model(dataloaders, num_images=25)

In [14]:
vgg_model.eval_model(dataloaders, mode = 'test')

Evaluating model on Test set
----------
Test batch 0/79
Evaluation completed in 9m 19s
Average Test loss     : 0.2411
Average Test accuracy : 0.9407
----------
